In [1]:
import base64
import json
import os
import random
import sys

import google.auth
from google.cloud import aiplatform
from google.cloud.aiplatform import gapic as aip
from google.cloud.aiplatform import hyperparameter_tuning as hpt
from google.protobuf.json_format import MessageToDict

In [7]:
PROJECT_ID='pytorch-tpu-nfs'
BUCKET_NAME='automl-samples'
CUSTOM_PREDICTOR_IMAGE_URI='gcr.io/pytorch-tpu-nfs/pt_tmp_forecast'

In [3]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

In [16]:
VERSION = 1
model_display_name = f"pt_tmp_forecast"
model_description = "PyTorch Forecast TFT Model"

MODEL_NAME = "pt_tmp_forecast"
health_route = "/ping"
predict_route = f"/predictions/{MODEL_NAME}"
serving_container_ports = [7080]

In [17]:
model = aiplatform.Model.upload(
    display_name=model_display_name,
    description=model_description,
    serving_container_image_uri=CUSTOM_PREDICTOR_IMAGE_URI,
    serving_container_predict_route=predict_route,
    serving_container_health_route=health_route,
    serving_container_ports=serving_container_ports,
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/64701051322/locations/us-central1/models/8965967568699392000/operations/4817968762168803328
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/64701051322/locations/us-central1/models/8965967568699392000
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/64701051322/locations/us-central1/models/8965967568699392000')


In [20]:
endpoint_display_name = f"pt_tmp_forecast-endpoint"
endpoint = aiplatform.Endpoint.create(display_name=endpoint_display_name)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/64701051322/locations/us-central1/endpoints/1079887544243453952/operations/2626404593499635712
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/64701051322/locations/us-central1/endpoints/1079887544243453952
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/64701051322/locations/us-central1/endpoints/1079887544243453952')


In [21]:
traffic_percentage = 100
machine_type = "n1-standard-16"
deployed_model_display_name = model_display_name
min_replica_count = 1
max_replica_count = 3
sync = True

model.deploy(
    endpoint=endpoint,
    deployed_model_display_name=deployed_model_display_name,
    machine_type=machine_type,
    #traffic_percentage=traffic_percentage,
    traffic_split=None,
    sync=sync,
)

INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/64701051322/locations/us-central1/endpoints/1079887544243453952
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/64701051322/locations/us-central1/endpoints/1079887544243453952/operations/5247499576629264384
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/64701051322/locations/us-central1/endpoints/1079887544243453952


resource name: projects/64701051322/locations/us-central1/endpoints/1079887544243453952

In [41]:
from google.cloud import bigquery

project_id = 'pytorch-tpu-nfs'
dataset_id = 'view_dataset'
table_id = 'weather_time_series_named'
location = 'us-central1'
staging_bucket = 'automl-samples'
experiment_prefix = 'pytorch-forecasting'

sql = f"""
SELECT *
FROM  `{project_id}.{dataset_id}.{table_id}`
LIMIT 122
"""
client = bigquery.Client(location=location, project=project_id)
query_job = client.query(
  sql,
  # Location must match that of the dataset(s) referenced in the query.
  location=location,
)  # API request - starts the query

data = query_job.to_dataframe()
data = data.drop(columns=['Date_Time'])
sample = data.to_dict()
test_data = {'data': sample}


import json

test_data_json = json.dumps({"instances": [test_data]})
#test_data_json = json.dumps([test_data])

model_name='pt_tmp_forecast'
import requests
headers = {"content-type": "application/json"}
for _ in range(1):
    json_response = requests.post(f'http://localhost:7080/predictions/{model_name}', data=test_data_json, headers=headers)
    print(json_response.text)

{"predictions": [[[-6.839053153991699, -6.9848246574401855, -2.4562153816223145, -3.1393628120422363, -2.8495378494262695, -2.9547688961029053, -3.8950858116149902, -3.958984136581421, -5.280339241027832, -5.265453815460205, -5.643465042114258, -5.765682697296143, -6.447474002838135, -6.877976417541504, -7.253215789794922, -7.215277671813965, -7.322080135345459, -7.348230361938477, -6.279671669006348, -6.468111515045166, -6.686850547790527, -6.77770471572876, -6.858955383300781, -7.053863525390625], [-6.914965629577637, -2.3017570972442627, -3.064727306365967, -2.8028132915496826, -2.92384672164917, -3.9014835357666016, -3.968947410583496, -5.344447135925293, -5.329333305358887, -5.72353458404541, -5.850763320922852, -6.55748176574707, -7.003578186035156, -7.39267635345459, -7.353636741638184, -7.464425563812256, -7.491635322570801, -6.38612174987793, -6.581673622131348, -6.808745384216309, -6.903045654296875, -6.987245559692383, -7.189067840576172, -7.3768229484558105], [-3.5739676952

In [54]:
#test_instance = {"instances": [{"data": {"b64": f"{str(b64_encoded.decode('utf-8'))}"}}]}
#{"instances":[{'data': json.dumps(sample)}]}

In [58]:
#dir(endpoint)
endpoint.list_models()

[id: "2722610692698931200"
model: "projects/64701051322/locations/us-central1/models/8965967568699392000"
display_name: "pt_tmp_forecast"
create_time {
  seconds: 1636743459
  nanos: 926758000
}
dedicated_resources {
  machine_spec {
    machine_type: "n1-standard-16"
  }
  min_replica_count: 1
  max_replica_count: 1
}
]

In [59]:
#prediction = endpoint.predict(instances={"instances":[{'data': json.dumps(sample)}]})
#prediction = endpoint.predict(instances=[{'data': json.dumps(sample)}])

In [42]:
# import google.auth
# import google.auth.transport.requests
# creds, project = google.auth.default()

# # creds.valid is False, and creds.token is None
# # Need to refresh credentials to populate those

# auth_req = google.auth.transport.requests.Request()
# creds.refresh(auth_req)
# model_name='pt_tmp_forecast'
# import requests
# headers = {"content-type": "application/json", "Authorization": f"Bearer {creds.token}"}
# for _ in range(1):
#     json_response = requests.post(f'https://us-central1-aiplatform.googleapis.com/v1/projects/${PROJECT_ID}/locations/us-central1/endpoints/${ENDPOINT_ID}', data=test_data_json, headers=headers)
#     print(json_response.text)

In [38]:
ENDPOINT_ID=1079887544243453952
PROJECT_ID='pytorch-tpu-nfs'

In [45]:
with open('test_sample.json', 'w') as f:
    f.write(test_data_json)

In [48]:
!curl \
-X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://us-central1-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/us-central1/endpoints/{ENDPOINT_ID}:predict \
-d "@test_sample.json"

{
  "predictions": [
    [
      [
        -0.29288816452026373,
        -0.31888580322265619,
        -0.25240516662597662,
        -0.2158665657043457,
        -5.4028177261352539,
        -0.18531084060668951,
        -0.20907926559448239,
        -0.19388437271118161,
        -0.16332101821899411,
        -0.23616600036621091,
        -0.14554452896118161,
        -0.35846328735351562,
        -0.32310914993286127,
        -0.31539392471313482,
        -0.25215387344360352,
        -4.85932731628418,
        -0.36718368530273438,
        -0.178004264831543,
        -0.20507144927978521,
        -0.20117282867431641,
        -0.15401315689086911,
        -0.1605944633483887,
        -0.2156224250793457,
        -0.187103271484375
      ],
      [
        -0.281463623046875,
        -0.21218538284301761,
        -0.17242574691772461,
        -5.453589916229248,
        -0.13939571380615229,
        -0.16272592544555661,
        -0.14703655242919919,
        -0.1161155700683594,
     

In [47]:
#test_data_json